# Импорт необходимых модулей

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
MEDIUM_SIZE = 20
BIGGER_SIZE = 30

plt.rc('font', size=BIGGER_SIZE)
plt.rc('axes', titlesize=MEDIUM_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=MEDIUM_SIZE)
plt.rc('ytick', labelsize=MEDIUM_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE)
plt.rc('text', color='green')

plt.rc('xtick', color='green')
plt.rc('ytick', color='green')

plt.rc('axes', labelcolor='green')
plt.rc('axes', labelcolor='green')

In [3]:
pd.set_option('display.max_colwidth', -1)

# На всякий случай...

[ISBN](https://en.wikipedia.org/wiki/International_Standard_Book_Number)

# Чтение данных

In [4]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [5]:
books = pd.read_csv('data/BX-Books.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

In [6]:
users = pd.read_csv('data/BX-Users.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251', low_memory=False)

# Знакомство

Ок, загрузили, надо теперь поглядеть, что это вообще за данные.

In [7]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User-ID        1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton Company,http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg


Последние три признака не являются информативными, поскольку представляют собой ссылки на картинки обложек книг разных размеров. Их можно удалить:

In [10]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [11]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
ISBN                   271379 non-null object
Book-Title             271379 non-null object
Book-Author            271378 non-null object
Year-Of-Publication    271379 non-null object
Publisher              271377 non-null object
dtypes: object(5)
memory usage: 10.4+ MB


Гдянем теперь, нет ли каких-нибудь повторений среди этих книг:

In [12]:
books_groupped = books.groupby('Book-Title').agg([np.array, len])

In [13]:
books_groupped.sort_values(by=[('ISBN', 'len')], ascending=False).head(20)

ISBN  \
                                                                                                                                                                                                                                                                                                                                                               array   
Book-Title                                                                                                                                                                                                                                                                                                                                                             
Selected Poems                  [081120958X, 0811201465, 0679750800, 0060931744, 0517101548, 0520012984, 0871401541, 0060909897, 0571050190, 1550651498, 0333516265, 0679430830, 0940262193, 0802151027, 1852240954, 0679765654, 0906427851, 0520069358, 0156003961, 0395320461, 006093168X, 0679724834, 0394702956, 0811209512, 0670809179, 0460871501, 0571210767]   
Little Women                    [185854176X, 0861782356, 0590203509, 0440447682, 1550134140, 0451529308, 0451518144, 0590437976, 0195813898, 0785788727, 0448110199, 0812523334, 0394562798, 1557042160, 0590433423, 0448054663, 0448023644, 0590404989, 0681400552, 1593081081, 1568651198, 0553211153, 0448172569, 0590225375]                                       
Wuthering Heights               [0553212583, 0553211412, 0812505166, 0553210211, 0439228913, 1566193087, 0590460307, 0140171002, 0880299185, 0451519582, 0706430131, 0141305479, 0451521196, 0817221999, 0894714805, 0395051029, 0895771594, 0312138946, 1570421161, 0804900116, 0451516508]                                                                           
The Secret Garden               [0812505018, 0590433466, 006440188X, 0590471724, 0440977096, 0879236493, 055321201X, 0439099390, 0440777062, 0812519108, 0451525817, 0451524179, 0397321651, 0451520807, 0440477093, 1569874115, 0453008402, 1569873925, 0590407201, 0340611839]                                                                                       
Dracula                         [156619301X, 055321148X, 9875021245, 0140052801, 0451175816, 0880299010, 8481301183, 1562542621, 0515053473, 0863221432, 0453007864, 0439101344, 8401492009, 0312863586, 0893757829, 014017186X, 1404336540, 8448700058, 0812504429, 0573608229]                                                                                       
Adventures of Huckleberry Finn  [0835902242, 059043389X, 0451519124, 0451523733, 0448110008, 0520237714, 0590408011, 0448058006, 0679448896, 0449912728, 0804900043, 0395051142, 0810200627, 0451516737, 0881421073, 0448149222, 0142500984, 1572701110, 0060806125, 0060467215]                                                                                       
Jane Eyre                       [0451518845, 0451523326, 1566193028, 156619024X, 8426102069, 0553210203, 0460012878, 0141305371, 2253004359, 0812523377, 0809598949, 3548301231, 0517668432, 1853268348, 2266083252, 0451515560, 0582528259, 1590071212, 843761449X]                                                                                                   
The Night Before Christmas      [0505523183, 0837885043, 155858465X, 0307109739, 1569872848, 0895424983, 030714190X, 0307102025, 0517185210, 0671684086, 0843134453, 0694004243, 0060283807, 0689840535, 0786806087, 0307137503, 0505521474, 0394807413]                                                                                                               
Pride and Prejudice             [055321215X, 0553213105, 0451523652, 1566190932, 0451525884, 0451519167, 0893756113, 0553210181, 0192815032, 0140238212, 0812523369, 0804900019, 0451520750, 0451516621, 0451522265, 0393997715, 0582529131, 3190025355]                                                                                                               
Great Expectations              

Очевидно есть повторения.

Заметно, что по некоторым публикациям не указан год издания, также по некоторым книгам имена авторов по-разному указаны.

В остальном это разные издания одних и тех же работ, напечатанных в разные (не все) годы разными издательствами.

Пока не совсем понятно, что делать с этой информацией.

Попробуем глянуть, какие есть оценки для разных изданий какого-нибудь произведения:

In [14]:
ratings[ratings.ISBN.isin(books_groupped.loc['Jane Eyre', ('ISBN', 'array')])][ratings['Book-Rating'] > 0]['User-ID'].nunique()

C:\Users\Efim_Golovin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


38

Итак, в частности для произведения **"Джейн Эйр"** присутствует довольно большое количество оценок от разных пользователей (для разных ISBN).

Собственно, логично предположить, что, оценивая книгу, читатель оценивает далеко не только само произведение, но еще и обложку, шрифт, цвет, качество бумаги (если это печатное издание).

Существуют, наверное и другие факторы, не имеющие отношения к самому произведению, однако здесь это особо не проверить, поскольку данных на эту тему нет.

In [15]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
ISBN                   271379 non-null object
Book-Title             271379 non-null object
Book-Author            271378 non-null object
Year-Of-Publication    271379 non-null object
Publisher              271377 non-null object
dtypes: object(5)
memory usage: 10.4+ MB


In [16]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [17]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
User-ID     278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


Итак, у нас есть 271379 книг и 278858 пользователей, по книгам известны их:
- ISBN;
- название;
- автор;
- год выпуска;
- издательство.

По пользователям (не по всем) известно их:
- местоположение;
- возраст.

Надо глянуть какой процент пользователей указали возраст:

In [ ]:
100 * len(users[~users.Age.isnull()]) / len(users)

Ок, ~60% пользователей не стесняются, чего не скажешь об остальных.

Глянем, как читатели распределены по возрасту:

In [ ]:
plt.figure(figsize=(35, 15))
n, bins, _ = plt.hist(users[~users.Age.isnull()].Age, bins=20)
plt.xticks(list(map(int, bins)))
plt.show()

Что же, очевидно, есть некоторая часть пользователей с возрастом, указанным не вполне адекватно:

In [ ]:
users[~users.Age.isnull()].Age.min()

In [ ]:
users[~users.Age.isnull()].Age.max()

Вряд ли кто-то в возрасте 0 или 244 лет что-то покупал из книг.

Заметим, что поле Location состоит из трех пунктов: город, регион, страна.

Надо бы разделить это поле на три, чтобы глянуть, как пользователи по миру раскиданы.

Перед этим надо глянуть, можно ли вообще легко это разделение сделать:

In [ ]:
locations = users.Location.values
locations = np.array([loc.split(', ') for loc in locations])

In [ ]:
incorrect_items_more = [ind for ind, item in enumerate(locations) if len(item) > 3]
incorrect_items_less = [ind for ind, item in enumerate(locations) if len(item) < 3]

In [ ]:
print(100 * (len(incorrect_items_more) + len(incorrect_items_less)) / len(locations))

~2% людей с коряво указанными данными по местоположению.

Надо глянуть, сколько из них оценили какие-нибудь книги:

In [ ]:
users_with_their_ratings_l = pd.merge(users[users.index.isin(incorrect_items_less)], ratings, how='inner', on='User-ID')

In [ ]:
plt.figure(figsize=(16, 16))
users_with_their_ratings_l['Book-Rating'].value_counts().plot.pie(autopct='%.2f')
plt.show()

In [ ]:
users_with_their_ratings_m = pd.merge(users[users.index.isin(incorrect_items_more)], ratings, how='inner', on='User-ID')

In [ ]:
plt.figure(figsize=(8, 8))
users_with_their_ratings_m['Book-Rating'].value_counts().plot.pie(autopct='%.2f')
plt.show()

Здесь графики приводятся в разных масштабах, чтобы не ввести в заблуждение по поводу объема выборок пользователей с коряво указанными локациями:

In [ ]:
print(len(incorrect_items_more), len(incorrect_items_less))

Надо бы еще глянуть, как много ISBN, указанных в таблице BX-Book-Ratings, отсутствуют в таблице BX-Books.

То же самое по поводу пользователей:

In [ ]:
ratings[~ratings.ISBN.isin(books.ISBN)].ISBN.nunique()

Получается, что в списке книг есть такие, о которых, вообще говоря, ничего не известно.

Надо глянуть, по скольким таким книгам есть оценки:

In [ ]:
non_zero_ratings = ratings[ratings['Book-Rating'] > 0]
non_zero_ratings[~non_zero_ratings.ISBN.isin(books.ISBN)].ISBN.nunique()

Что же, у нас есть оценки пользователей по книгам, о которых мы ничего не знаем.

Глянем, как это количество соотносится с общим количеством оцененных пользователями книг:

In [ ]:
100 * non_zero_ratings[~non_zero_ratings.ISBN.isin(books.ISBN)].ISBN.nunique() / non_zero_ratings.ISBN.nunique()

С одной стороны 19% - это довольно много, с другой - это 19% оценок книг, о которых ничего неизвестно.

Это просто какие-то номера, по которым нельзя будет, скажем построить какой-нибудь Item-based алгоритм, после применения Collaborative filtering нельзя будет сказать, каким именно книгам были восстановлены оценки, нельзя будет сделать никаких выводов по этому поводу.

Собственно поэтому логичнее эти оценки пока не рассматривать.

In [ ]:
non_zero_ratings = non_zero_ratings[non_zero_ratings.ISBN.isin(books.ISBN)]

In [ ]:
non_zero_ratings.ISBN.nunique()

In [ ]:
non_zero_ratings[~non_zero_ratings['User-ID'].isin(users['User-ID'])]['User-ID'].nunique()

Итак, нет ни одного ID для которого бы в таблице users не было бы записи.

Хорошо, теперь надо еще ответить на обратные вопросы: есть ли такие пользователи, которые не дали оценки ни одной книге и есть ли такие книги, которые не были оценены ни одним пользователем:

In [ ]:
100 * users[~users['User-ID'].isin(non_zero_ratings['User-ID'])]['User-ID'].nunique() / users['User-ID'].nunique()

In [ ]:
100 * books[~books.ISBN.isin(non_zero_ratings.ISBN)].ISBN.nunique() / books.ISBN.nunique()

Получается, у нас есть ~75.6% пользователей, никак ни одной книги не оценивших и ~44.8% книг, никах никем не оценённых.

# К вопросу о правильном Evaluation

Здесь общая идея следующая:
- сформировать выборку троек (пользователь, книга, оценка), желательно, чтобы эта выборка покрыла как можно больше книг;
- извлечь эти тройки из основного набора данных (оценок) и далее считать, что для выбранных пользователей и книг оценки неизвестны и должны быть восстановлены;
- собственно, применить один из алгоритмов восстановления оценок;
- пользуясь выбраннной метрикой (RMSE или NDCG) оценить качество восстановления пользовательских оценок.

# Сколько читателей поставили оценки и какое количество книг было оценено

Ок, теперь надо глянуть, какие пользователи сколько раз поставили оценки и какие книги сколько раз были оценены.

Это необходимо для того, чтобы понять как разделять датасет на трейн и тест.

Также нужно понять какую часть пользователей и книг нужно отсеять (пользователи, оценившие слишком мало книг из всего множества, книги, оцененные слишком мало пользователями).

In [ ]:
non_zero_ratings_groupped_by_user = non_zero_ratings.groupby('User-ID').agg(len)

In [ ]:
non_zero_ratings_groupped_by_user.sort_values(by=['ISBN'], ascending=False).head(10)

In [ ]:
len(non_zero_ratings_groupped_by_user[non_zero_ratings_groupped_by_user.ISBN > 10])

In [ ]:
non_zero_ratings_groupped_by_user[non_zero_ratings_groupped_by_user.ISBN > 10].index.values

In [ ]:
non_zero_ratings[non_zero_ratings['User-ID'] == 278633].index.values

In [ ]:
len(non_zero_ratings_groupped_by_user)

In [ ]:
non_zero_ratings_groupped_by_isbn = non_zero_ratings.groupby('ISBN').agg(len)

In [ ]:
non_zero_ratings_groupped_by_isbn.sort_values(by=['User-ID'], ascending=False).head(10)

In [ ]:
len(non_zero_ratings_groupped_by_isbn[non_zero_ratings_groupped_by_isbn['User-ID'] > 10])

In [ ]:
len(non_zero_ratings_groupped_by_isbn)

# Итог

Итого на данный момент можно сделать несколько выводов по данным, а именно:
- среди книг присутствует большое количество повторений (одни и те же произведения, но от разных издательств, разных годов издания, одинаково названные, но вообще говоря, разные произведения и т.д.)
- ~ у 40% пользователей не указан возраст;
- некоторая часть пользователей указала возраст неправильно (значения типа 0 и 244);
- за исключением ~2% пользователей местоположение можно разделить на город, регион и страну;
- среди этих 2% бОльшая часть пользоваталей не поставила оценок ни одной книге;
- ~19% оценок относятся к книгам, о которых ничего неизвестно (их нельзя найти по ISBN в таблице BX-Books.csv);
- ~75.6% пользователей либо только заходили на страницу с книгой, но не поставили оценку, либо вообще ничего не делали;
- ~44.8% книг либо были только просмотрены, но не оценены, либо вообще никак не были задействованы